1. Используйте линейную интерполяцию для температуры воздуха, облачности 
    и точки росы
2. Вычислите первую производную для температуры воздуха, запишите ее в столбец
    air_temperature1
3. Вычислите синус и косинус от угла направления ветра, запишите их в столбцы
    wind_direction_cos, wind_direction_sin
4. Удалите столбцы site_id, building_id, primary_use, square_feet, year_build,
    floor_count, wind_direction - получилось
5. Удалите из данных строки с meter_reading = 0 - получилось

In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [32]:
data = pd.read_csv('energy_2.csv')
print (data.head())

             timestamp  site_id  building_id  meter  meter_reading  \
0  2016-01-01 00:00:00        0            0      0            0.0   
1  2016-01-01 01:00:00        0            0      0            0.0   
2  2016-01-01 02:00:00        0            0      0            0.0   
3  2016-01-01 03:00:00        0            0      0            0.0   
4  2016-01-01 04:00:00        0            0      0            0.0   

  primary_use  square_feet  year_built  floor_count  air_temperature  \
0   Education         7432      2008.0          NaN             25.0   
1   Education         7432      2008.0          NaN             24.4   
2   Education         7432      2008.0          NaN             22.8   
3   Education         7432      2008.0          NaN             21.1   
4   Education         7432      2008.0          NaN             20.0   

   cloud_coverage  dew_temperature  precip_depth_1_hr  sea_level_pressure  \
0             6.0             20.0                0.0              10

In [33]:
# столбцы wind_direction_cos, wind_direction_sin

data['wind_direction_rad'] = data['wind_direction'] * np.pi / 100
data['wind_direction_sin'] = np.sin(data['wind_direction_rad'])
data['wind_direction_cos'] = np.cos(data['wind_direction_rad'])
data = data.drop(labels = ['wind_direction_rad'], axis = 1)

In [34]:
print(data.head())

             timestamp  site_id  building_id  meter  meter_reading  \
0  2016-01-01 00:00:00        0            0      0            0.0   
1  2016-01-01 01:00:00        0            0      0            0.0   
2  2016-01-01 02:00:00        0            0      0            0.0   
3  2016-01-01 03:00:00        0            0      0            0.0   
4  2016-01-01 04:00:00        0            0      0            0.0   

  primary_use  square_feet  year_built  floor_count  air_temperature  \
0   Education         7432      2008.0          NaN             25.0   
1   Education         7432      2008.0          NaN             24.4   
2   Education         7432      2008.0          NaN             22.8   
3   Education         7432      2008.0          NaN             21.1   
4   Education         7432      2008.0          NaN             20.0   

   cloud_coverage  dew_temperature  precip_depth_1_hr  sea_level_pressure  \
0             6.0             20.0                0.0              10

In [35]:
# производная по температуре воздуха

data['air_temperature1'] = data['air_temperature'].diff()
data.at[0, 'air_temperature1'] = data.at[1, 'air_temperature1']

In [36]:
# удаление пустых значений

data = data.loc[data['meter_reading'] > 0]

In [37]:
# уадаление ненужных столбцов

data = data.drop(labels = ['site_id', 'building_id', 'meter', 'primary_use', 'square_feet',
                         'year_built', 'floor_count', 'wind_direction'], axis = 1)
print(data.head())

                timestamp  meter_reading  air_temperature  cloud_coverage  \
704   2016-01-30 08:00:00        43.6839              8.3             0.0   
725   2016-01-31 05:00:00        37.5408             12.8             0.0   
737   2016-01-31 17:00:00        52.5571             20.6             0.0   
2366  2016-04-08 14:00:00        59.3827             21.7             2.0   
2923  2016-05-01 19:00:00       448.0000             31.1             0.0   

      dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_speed  \
704               6.1                0.0              1019.0         2.1   
725              10.0                0.0              1021.9         0.0   
737              11.7                0.0              1020.9         1.5   
2366             14.4                0.0              1015.1         3.1   
2923             17.2                0.0              1016.1         4.1   

      wind_direction_sin  wind_direction_cos  air_temperature1  
704         5.8

In [38]:
data.to_csv('energy_data_1.csv', index = False)

In [42]:
# РАЗДЕЛЕНИЕ ДАННЫХ 
# на обучающую и проверочные выборки, обычно 90/10, 80/20, 75/25, 60/40

data_train, data_test = train_test_split(data, test_size = 0.20)
print(data_train.head())

                timestamp  meter_reading  air_temperature  cloud_coverage  \
5947  2016-09-04 19:00:00        234.801             32.2             0.0   
6336  2016-09-21 00:00:00        237.531             24.4             6.0   
3632  2016-05-31 08:00:00        178.831             21.7             0.0   
6655  2016-10-04 07:00:00        249.135             24.4             0.0   
8532  2016-12-21 12:00:00        229.340             15.0             8.0   

      dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_speed  \
5947             21.7                0.0              1017.4         2.6   
6336             22.8               61.0              1014.0         2.6   
3632             20.6                0.0              1013.1         0.0   
6655             22.2                0.0              1011.2         2.1   
8532             13.9                0.0              1023.3         3.6   

      wind_direction_sin  wind_direction_cos  air_temperature1  
5947           

In [44]:
# ПОСТРОЕНИЕ МОДЕЛИ ПО ОБУЧАЮЩЕЙ ВЫБОРКЕ
# Среднее значение

model = float(data_train['meter_reading'].mean())
print('{0:.4}'.format(model))

237.9


ПРОВЕРКА МОДЕЛИ
на проверочной выборке (с известным проверочным значением)

In [57]:
err = np.mean(np.abs(data_test['meter_reading'] - model)) 
print('{0:.4}%'.format(100*err/model))

16.12%


Задание
1. Выделите час из timestamp
2. Вычислите среднее значение энергопотребления по часам на обучающей выборке 
3. Проверте точность работы модели "среднее по часам" относительно модели "общее среднее" на проверочной выборке

Выделение времени

преобразование метки времени в дату и выделение из даты часа суток

In [53]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['hour'] = data['timestamp'].dt.hour

In [55]:
# сохранение данных

data.to_csv('energy_data_2.csv', index = False)
print(data.head())

               timestamp  meter_reading  air_temperature  cloud_coverage  \
704  2016-01-30 08:00:00        43.6839              8.3             0.0   
725  2016-01-31 05:00:00        37.5408             12.8             0.0   
737  2016-01-31 17:00:00        52.5571             20.6             0.0   
2366 2016-04-08 14:00:00        59.3827             21.7             2.0   
2923 2016-05-01 19:00:00       448.0000             31.1             0.0   

      dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_speed  \
704               6.1                0.0              1019.0         2.1   
725              10.0                0.0              1021.9         0.0   
737              11.7                0.0              1020.9         1.5   
2366             14.4                0.0              1015.1         3.1   
2923             17.2                0.0              1016.1         4.1   

      wind_direction_sin  wind_direction_cos  air_temperature1  hour  
704         5.8

In [71]:
data = pd.read_csv('energy_data_2.csv')
print(data.head())

             timestamp  meter_reading  air_temperature  cloud_coverage  \
0  2016-01-30 08:00:00        43.6839              8.3             0.0   
1  2016-01-31 05:00:00        37.5408             12.8             0.0   
2  2016-01-31 17:00:00        52.5571             20.6             0.0   
3  2016-04-08 14:00:00        59.3827             21.7             2.0   
4  2016-05-01 19:00:00       448.0000             31.1             0.0   

   dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_speed  \
0              6.1                0.0              1019.0         2.1   
1             10.0                0.0              1021.9         0.0   
2             11.7                0.0              1020.9         1.5   
3             14.4                0.0              1015.1         3.1   
4             17.2                0.0              1016.1         4.1   

   wind_direction_sin  wind_direction_cos  air_temperature1  hour  
0        5.877853e-01        8.090170e-01       

In [68]:
# вычисление стандартного отклонения для всех значений

std = np.std(data['meter_reading'])
print('{0:.4}'.format(std))

49.13


Построение базовой модели

#среднее значение и его ошибка

In [69]:
model = float(data['meter_reading'].mean())
err = np.mean(np.abs(data['meter_reading'] - model))
print ('Модель: {0:.4},'.format(model),
'Ошибка модели: {0:.4}%'.format(100*err/model),
'Ошибка данных: {0:.4}%'.format(100*std/model))

Модель: 237.7, Ошибка модели: 15.17% Ошибка данных: 20.66%


Модель по первому часу
Выберем значения hour == 0

In [73]:
data0 = data.loc[data['hour'] == 0]
print (data0.head())

               timestamp  meter_reading  air_temperature  cloud_coverage  \
11   2016-05-21 00:00:00        249.817             26.7             6.0   
35   2016-05-22 00:00:00        251.182             28.9             2.0   
59   2016-05-23 00:00:00        237.531             26.7             6.0   
83   2016-05-24 00:00:00        223.197             26.1             4.0   
107  2016-05-25 00:00:00        208.863             26.1             4.0   

     dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_speed  \
11              19.4                0.0              1017.5         0.0   
35              17.2                0.0              1013.6         4.1   
59              14.4                0.0              1011.5         7.2   
83              15.6                0.0              1014.1         6.2   
107             13.9                0.0              1019.6         5.1   

     wind_direction_sin  wind_direction_cos  air_temperature1  hour  
11         0.000000e+0

Построим модель по часу и сравним ее точность с базовой моделью

In [75]:
model0 = float(data0['meter_reading'].mean())
err = np.mean(np.abs(data0['meter_reading'] - model0))
print ('Модель: {0:.4},'.format(model0),
'Ошибка модели: {0:.4}%'.format(100*err/model0),
'Ошибка данных: {0:.4}%'.format(100*std/model0))

Модель: 238.7, Ошибка модели: 13.63% Ошибка данных: 20.58%
